sgdclassifier | bag of sites | 0.85722

csr_matrix заменён на CountVectorizer; local проверка cross_val_score + TimeSeriesSplit

In [8]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import os
import pickle
import numpy as np
import pandas as pd
# from scipy.sparse import csr_matrix
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer

# PATH_TO_DATA = '../capstone_user_identification'
PATH_TO_DATA = '../input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2'

In [ ]:
# !unzip capstone_user_identification.zip

In [2]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                       index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                      index_col='session_id')
train_df.shape, test_df.shape

((253561, 21), (82797, 20))

In [3]:
train_df.sort_values(by='time1', inplace=True)

**Пока для прогноза будем использовать только индексы посещенных сайтов. Индексы нумеровались с 1, так что заменим пропуски на нули.**

In [4]:
train_test_df = pd.concat([train_df, test_df])
train_test_df_sites = train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')

In [ ]:
!mkdir tmp

In [5]:
train_test_df_sites[:253561].to_csv('tmp/train.csv', header=False, index=False, sep=' ')
train_test_df_sites[253561:].to_csv('tmp/test.csv', header=False, index=False, sep=' ')

In [6]:
cv = CountVectorizer()
with open('tmp/train.csv') as train:
    X_train_sparse = cv.fit_transform(train)
with open('tmp/test.csv') as test:
    X_test_sparse = cv.transform(test)
y = train_df['target']

**Создайте объект `sklearn.linear_model.SGDClassifier` с логистической функцией потерь и параметром *random_state*=17. Остальные параметры оставьте по умолчанию, разве что *n_jobs*=-1 никогда не помешает. Обучите  модель на выборке `(X_train, y_train)`.**

In [7]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)

In [9]:
%%time
time_split = TimeSeriesSplit(n_splits=10)

cv_scores = cross_val_score(sgd_logit, X_train_sparse, y, cv=time_split, scoring='roc_auc')

CPU times: user 4.19 s, sys: 10.4 ms, total: 4.2 s
Wall time: 3.87 s


In [10]:
cv_scores, cv_scores.mean()

(array([0.78397985, 0.62833329, 0.86997686, 0.94582411, 0.85014222,
        0.89251879, 0.92898312, 0.85348877, 0.92329172, 0.89561549]),
 0.8572154229480724)

**Сделайте прогноз в виде предсказанных вероятностей отнесения к классу 1 для тестовой выборки с помощью той же *sgd_logit*, обученной уже на всей обучающей выборке (а не на 70%).**

In [ ]:
sgd_logit.fit(X_train_sparse, y)
logit_test_pred_proba = sgd_logit.predict_proba(X_test_sparse)

**Запишите ответы в файл и сделайте посылку на Kaggle. Дайте своей команде (из одного человека) на Kaggle говорящее название – по шаблону "[YDF & MIPT] Coursera_Username", чтоб можно было легко идентифицировать Вашу посылку на [лидерборде](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard/public).**

**Результат, который мы только что получили, соответствует бейзлайну "SGDCLassifer" на лидерборде, задача на эту неделю – как минимум его побить.**

In [ ]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
write_to_submission_file(logit_test_pred_proba[:, 1], 'spec_submission1.csv') # leaderboard 0.91646

Бейзлайн sgd_logit_benchmark: 0.91273 побит